In [ ]:
def analyse_numeric_mean_stat(df, target, alpha=0.05):
    # df - датасет
    # target - целевая переменная
    # alpha - уровень значимости
    # для бинарного таргета
    
    confirmed = {}
    rejected = {}
    df = df.select_dtypes(include='number')
    colslist = df.columns.to_list()
    colslist.remove(target)
    
    for i in colslist:
        print(f'Проверяем признак {i}')
        print('Нулевая гипотеза: средние значения в выборках по классам целевой переменной равны')
        print('Альтернативная гипотеза: средние значения в выборках по классам целевой переменной различаются')
        print(f'Распределение признака {i} по классам')
        spread = df[[i, target]]
        spread_1 = spread.loc[spread[target] == 1, i]
        spread_0 = spread.loc[spread[target] == 0, i]
        sns.kdeplot(spread_0, shade=True, label = 'Target 0', color='g')
        sns.kdeplot(spread_1, shade=True, label = 'Target 1', color='r')
        plt.xlabel(i)
        plt.title(f'{i} grouped by target variable')
        plt.show()
        print('Проверка на нормальность: ')
        shap = shapiro(spread[i])
        if shap[1] < alpha:
            print(f'По полученному значению p-value, которое меньше {alpha}, можем заключить, что гипотеза о "нормальности" отвергается.')
            print(f'Мы не можем воспользоваться критерием Стьюдента для проверки гипотезы о равности мат. ожиданий признака {i}, воспользуемся непараметрическим аналогом, критерием Манна-Уитни')
            man = mannwhitneyu(spread_1, spread_0)
            if man[1] < alpha:
                print(f'Согласно значению p-value = {man[1]}, гипотеза о равности мат. ожиданий ОТВЕРГАЕТСЯ, но стоит ради дополнительной проверки обратиться к доверительным интервалам')
                plt.figure(figsize=(8, 5))

                sns.pointplot(x=target, y=i, data=spread, capsize=.1)

                plt.title(f'Confidence intervals (95 %) for {i}')
                plt.show()
                rejected[i] = man[1]
            else:
                print(f'Согласно значению p-value = {man[1]}, гипотеза о равности мат. ожиданий ПОДТВЕРЖДАЕТСЯ, но стоит ради дополнительной проверки обратиться к доверительным интервалам')
                plt.figure(figsize=(8, 5))

                sns.pointplot(x=target, y=i, data=spread, capsize=.1)

                plt.title(f'Confidence intervals (95 %) for {i}')
                plt.show()
                confirmed[i] = man[1]
        else:
            print(f'p-value, больше {alpha}, похоже, что гипотеза о "нормальности" подтверждается.')
            print(f'Мы можем воспользоваться критерием Стьюдента для проверки гипотезы о равности мат. ожиданий признака {i}')
            t = ttest_ind(spread_1, spread_0)
            if t[1] < alpha:
                print(f'Согласно значению p-value = {t[1]}, гипотеза о равности мат. ожиданий ОТВЕРГАЕТСЯ, но стоит ради дополнительной проверки обратиться к доверительным интервалам')
                plt.figure(figsize=(8, 5))

                sns.pointplot(x=target, y=i, data=spread, capsize=.1)

                plt.title(f'Confidence intervals (95 %) for {i}')
                plt.show()
                rejected[i] = t[1]
            else:
                print(f'Согласно значению p-value = {t[1]}, гипотеза о равности мат. ожиданий ПОДТВЕРЖДАЕТСЯ, но стоит ради дополнительной проверки обратиться к доверительным интервалам')
                plt.figure(figsize=(8, 5))

                sns.pointplot(x=target, y=i, data=spread, capsize=.1)

                plt.title(f'Confidence intervals (95 %) for {i}')
                plt.show()
                confirmed[i] = t[1]
    print('ЗАКЛЮЧЕНИЕ')
    print('Мы подтвердили, что в этих признаках разницы в мат ожиданиях нет.')
    for i in confirmed:
        print(f'{i} : {confirmed[i]}')
    print('*'*50)
    print('Мы опровергли, что в этих признаках разницы в мат ожиданиях нет. Стоит обратить на них пристальное внимание')
    for i in rejected:
        print(f'{i} : {rejected[i]}')
    